# "Text Preprocessing"
> "Vairous preprocessing steps required to clean and prepare your data in NLP"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [Text Preprocessing, Natural Language Processing]
- image: images/posts/text_preprocessing.jpg
- hide: false
- search_exclude: false

In any machine learning task, cleaning and pre-processing of the data is a very important step. The better we can represent our data, the better the model training and prediction can be expected.

Specially in the domain of Natural Language Processing (NLP) the data is unstructured. It become crucial to clean and properly format it based on the task at hand. There are various pre-processing steps that can be performed but not necessary to perform all. These steps should be applied based on the problem statement.

Example: Sentiment analysis on twitter data can required to remove hashtags, emoticons, etc. but this may not be the case if we are doing the same analysis on customer feedback data.

Here we are using the twitter_sample dataset from the nltk library.

In [ ]:
#collapse-hide
# Import libraries and load the data
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import demoji
import contractions
import unidecode
from num2words import num2words
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from spellchecker import SpellChecker

pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.display.max_colwidth=None

nltk.download('twitter_samples') # Download the dataset

# We are going to use the Negative and Positive Tweets file which each contains 5000 tweets.
for name in twitter_samples.fileids():
    print(f' - {name}')

 - negative_tweets.json
 - positive_tweets.json
 - tweets.20150430-223406.json


[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [ ]:
#collapse-hide

# Load the negative tweets file and assign label as 0 for negative
negative_tweets = twitter_samples.strings("negative_tweets.json")
df_neg = pd.DataFrame(negative_tweets, columns=['text'])
df_neg['label'] = 0

# Load the positive tweets file and assign label as 1 for positive
positive_tweets = twitter_samples.strings("positive_tweets.json")
df_pos = pd.DataFrame(positive_tweets, columns=['text'])
df_pos['label'] = 1

df = pd.concat([df_pos, df_neg]) # Concatenate both the files
df = df.sample(frac=1).reset_index(drop=True) # Shuffle the data to mix negative and positive tweets

In [ ]:
print(f'Shape of the whole data is: {df.shape[0]} rows and {df.shape[1]} columns')

Shape of the whole data is: 10000 rows and 2 columns


In [ ]:
# Look at the head of the dataframe
df.head()

,text,label
0,Username Changed! :D,1
1,@kimtaaeyeonss unnieeee!!!:),1
2,@amyewest Thanks! I hope you've got a good book to keep you company. :-),1
3,:) where are you situated? @Hijay09,1
4,"@egaroo You're welcome, I'm glad you liked it :)",1


> Note: Always make it a practice to first skim the dataset before performing any text pre-processing steps. It is important because text data can be very noisy eg. dates are written in different formats, present of accented characters, etc. These are stuff we can easily miss if we don't go through the dataset properly.

## Lower Casing

Lowercasing is a common text preprocessing technique. It helps to transform all the text in same case. <br>
Examples 'The', 'the', 'ThE' -> 'the'

This is also useful to find all the duplicates since words in different cases are treated as separate words and becomes difficult for us to remove redundant words in all different case combination.

This may not be helpful when we do tasks like Part of Speech tagging (where proper casing gives some information about Nouns and so on) and Sentiment Analysis (where upper casing refers to anger and so on)

In [ ]:
df.text = df.text.str.lower()
df.head(2)

,text,label
0,username changed! :d,1
1,@kimtaaeyeonss unnieeee!!!:),1


## Remove

### URL's

URL stands for Uniform Resource Locator. If present in a text, it represents the location of another website.

If we are performing any websites backlink analysis, in that case URL's are useful to keep. Otherwise, they don't provide any information. So we can remove them from our text.

In [ ]:
df.text = df.text.str.replace(r'https?://\S+|www\.\S+', '', regex=True)
df.head()

,text,label
0,username changed! :d,1
1,@kimtaaeyeonss unnieeee!!!:),1
2,@amyewest thanks! i hope you've got a good book to keep you company. :-),1
3,:) where are you situated? @hijay09,1
4,"@egaroo you're welcome, i'm glad you liked it :)",1


### E-mail

E-mail id's are common in customer feedback data and they do not provide any useful information. So we remove them from the text.

Twitter data that we are using does not contain any email id's. Hence, please find the code snipper with an dummy example to remove e-mail id's.

In [ ]:
text = 'I have being trying to contact xyz via email to xyz@abc.co.in but there is no response.'
re.sub(r'\S+@\S+', '', text)

'I have being trying to contact xyz via email to  but there is no response.'

### Date

Dates can be represented in various formats and can be difficult at times to remove them. They are unlikely to contain any useful information for predicting the labels.

Below I have used dummy text to showcase the following task.

In [ ]:
text = "Today is 22/12/2020 and after two days on 24-12-2020 our vacation starts until 25th.09.2021"

# 1. Remove date formats like: dd/mm/yy(yy), dd-mm-yy(yy), dd(st|nd|rd).mm/yy(yy)
re.sub(r'\d{1,2}(st|nd|rd|th)?[-./]\d{1,2}[-./]\d{2,4}', '', text)

'Today is  and after two days on  our vacation starts until '

In [ ]:
text = "Today is 11th of January, 2021 when I am writing this post. I hope to post this by February 15th or max to max by 20 may 21 or 20th-December-21"

# 2. Remove date formats like: 20 apr 21, April 15th, 11th of April, 2021
pattern = re.compile(r'(\d{1,2})?(st|nd|rd|th)?[-./,]?\s?(of)?\s?([J|j]an(uary)?|[F|f]eb(ruary)?|[Mm]ar(ch)?|[Aa]pr(il)?|[Mm]ay|[Jj]un(e)?|[Jj]ul(y)?|[Aa]ug(ust)?|[Ss]ep(tember)?|[Oo]ct(ober)?|[Nn]ov(ember)?|[Dd]ec(ember)?)\s?(\d{1,2})?(st|nd|rd|th)?\s?[-./,]?\s?(\d{2,4})?')
pattern.sub(r'', text)

'Today is  when I am writing this post. I hope to post this byor max to max by or '

There are various formats in which dates are represented and the above regex can be customized in many ways. Above, "byor" got combined cause we are trying multiple format in single regex pattern. You can customize the above expression accordingly to your need.

### HTML Tags

If we are extracting data from various websites, it is possible that the data also contains HTML tags. These tags does not provide any information and should be removed. These tags can be removed using regex or by using BeautifulSoup library.

In [ ]:
# Dummy text
text = """
<title>Below is a dummy html code.</title>
<body>
    <p>All the html opening and closing brackets should be remove.</p>
    <a href="https://www.abc.com">Company Site</a>
</body>
"""

In [ ]:
# Using regex to remove html tags
pattern = re.compile('<.*?>')
pattern.sub('', text)

'\nBelow is a dummy html code.\n\n    All the html opening and closing brackets should be remove.\n    Company Site\n\n'

In [ ]:
# Using Beautiful Soup
def remove_html(text):
    clean_text = BeautifulSoup(text).get_text()
    return clean_text

In [ ]:
remove_html(text)

'\nBelow is a dummy html code.\n\nAll the html opening and closing brackets should be remove.\nCompany Site\n\n'

### Emojis

As more and more people have started using social media emoji's play a very crucial role. Emoji's are used to express emotions that are universally understood.

In some analysis such as sentiment analysis emoji's can be useful. We can convert them to words or create some new features based on them. For some analysis we need to remove them. Find the below code snippet used to remove the emoji's.

In [ ]:
#collapse-hide
# Reference: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
text = "game is on 🔥🔥. Hilarious😂"
remove_emoji(text)

'game is on . Hilarious'

In [ ]:
# Remove emoji's from text
df.text = df.text.apply(lambda x: remove_emoji(x))

### Emoticons

Emoji's and Emoticons are different. Yes!!<br>
Emoticons are used to express facial expressions using keyboard characters such as letters, numbers, and pucntuation marks. Where emjoi's are small images.

Thanks to [Neel Shah](https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py) for curating a dictionary of emoticons and their description. We shall use this dictionary and remove the emoticons from our text.

In [ ]:
#collapse-hide

EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [ ]:
def remove_emoticons(text):
    emoticons_pattern = re.compile(u'(' + u'|'.join(emo for emo in EMOTICONS) + u')')
    return emoticons_pattern.sub(r'', text)

In [ ]:
remove_emoticons("Hello :->")

'Hello '

In [ ]:
# Remove emoticons from text
df.text = df.text.apply(lambda x: remove_emoticons(x))

### Hashtags and Mentions

We are habituated to use hashtags and mentions in our tweet either to indicate the context or bring attention to an individual. Hashtags can be used to extract features, to see what's trending and in various other applications.

Since, we don't require them we'll remove them.

In [ ]:
def remove_tags_mentions(text):
    pattern = re.compile(r'(@\S+|#\S+)')
    return pattern.sub('', text)

In [ ]:
text = "live @flippinginja on #younow - jonah and jareddddd"
remove_tags_mentions(text)

'live  on  - jonah and jareddddd'

In [ ]:
# Remove hashtags and mentions
df.text = df.text.apply(lambda x: remove_tags_mentions(x))

### Punctuations

Punctuations are character other than alphaters and digits. These include [!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~]

It is better remove or convert emoticons before removing the punctuations, since if we do the other we around we might loose the emoticons from the text. Another example, if the text contains $10.50 then we'll remove the .(dot) and the value will loose it's meaning.

In [ ]:
PUNCTUATIONS = string.punctuation

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCTUATIONS))

In [ ]:
df.text = df["text"].apply(lambda text: remove_punctuation(text))

,text,label
0,username changed d,1
1,unnieeee,1
2,thanks i hope youve got a good book to keep you company,1
3,where are you situated,1
4,youre welcome im glad you liked it,1


### Stopwords

Stopwords are commonly occuring words in any language. Such as, in english these words are 'the', 'a', 'an', & many more. They are in most cases not useful and should be removed.

There are certain tasks in which these words are useful such as Part-of-Speech(POS) tagging, language translation. Stopwords are compiled for many languages, for english language we can use the list from the nltk package.

In [ ]:
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in STOPWORDS])

In [ ]:
# Remove stopwords
df.text = df.text.apply(lambda text: remove_stopwords(text))
df.head()

,text,label
0,username changed,1
1,unnieeee,1
2,thanks hope youve got good book keep company,1
3,situated,1
4,youre welcome im glad liked,1


### Numbers

We may remove numbers if they are not useful in our analysis. But analysis in the financial domain, numbers are very useful.

In [ ]:
df.text = df.text.str.replace(r'\d+', '', regex=True)

### Extra whitespaces

After usually after preprocessing the text there might be extra whitespaces that might be created after transforming, removing various characters. Also, there is a need to remove all the new line, tab characters as well from our text.

In [ ]:
def remove_whitespaces(text):
    return " ".join(text.split())

In [ ]:
text = "  Whitespaces in the beginning are removed  \t as well \n  as in between  the text   "

clean_text = " ".join(text.split())
clean_text

'Whitespaces in the beginning are removed as well as in between the text'

In [ ]:
df.text = df.text.apply(lambda x: remove_whitespaces(x))

### Frequent words

Previously we have removed stopwords which are common in any language. If we are working in any domain, we can also remove the common words used in that domain which don't provide us with much information.

In [ ]:
def freq_words(text):
    tokens = word_tokenize(text)
    FrequentWords = []

    for word in tokens:
        counter[word] += 1

    for (word, word_count) in counter.most_common(10):
        FrequentWords.append(word)
    return FrequentWords

def remove_fw(text, FrequentWords):
    tokens = word_tokenize(text)
    without_fw = []
    for word in tokens:
        if word not in FrequentWords:
            without_fw.append(word)

    without_fw = ' '.join(without_fw)
    return without_fw

counter = Counter()

In [ ]:
text = """
Natural Language Processing is the technology used to aid computers to understand the human’s natural language. It’s not an easy task teaching machines to understand how we communicate. Leand Romaf, an experienced software engineer who is passionate at teaching people how artificial intelligence systems work, says that “in recent years, there have been significant breakthroughs in empowering computers to understand language just as we do.” This article will give a simple introduction to Natural Language Processing and how it can be achieved. Natural Language Processing, usually shortened as NLP, is a branch of artificial intelligence that deals with the interaction between computers and humans using the natural language. The ultimate objective of NLP is to read, decipher, understand, and make sense of the human languages in a manner that is valuable. Most NLP techniques rely on machine learning to derive meaning from human languages.
"""

In [ ]:
FrequentWords = freq_words(text)
print(FrequentWords)

[',', 'to', '.', 'is', 'the', 'understand', 'Natural', 'Language', 'Processing', 'computers']


In [ ]:
fw_result = remove_fw(text, FrequentWords)
fw_result

'technology used aid human ’ s natural language It ’ s not an easy task teaching machines how we communicate Leand Romaf an experienced software engineer who passionate at teaching people how artificial intelligence systems work says that “ in recent years there have been significant breakthroughs in empowering language just as we do. ” This article will give a simple introduction and how it can be achieved usually shortened as NLP a branch of artificial intelligence that deals with interaction between and humans using natural language The ultimate objective of NLP read decipher and make sense of human languages in a manner that valuable Most NLP techniques rely on machine learning derive meaning from human languages'

### Rare words

Rare words are similar to frequent words. We can remove them because they are so less that they cannot add any value to the purpose.

In [ ]:
def rare_words(text):
    # tokenization
    tokens = word_tokenize(text)
    for word in tokens:
        counter[word]= +1

    RareWords = []
    number_rare_words = 10
    # take top 10 frequent words
    frequentWords = counter.most_common()
    for (word, word_count) in frequentWords[:-number_rare_words:-1]:
        RareWords.append(word)

    return RareWords

def remove_rw(text, RareWords):
    tokens = word_tokenize(text)
    without_rw = []
    for word in tokens:
        if word not in RareWords:
            without_rw.append(word)

    without_rw = ' '.join(without_fw)
    return without_rw

counter = Counter()

In [ ]:
text = """
Natural Language Processing is the technology used to aid computers to understand the human’s natural language. It’s not an easy task teaching machines to understand how we communicate. Leand Romaf, an experienced software engineer who is passionate at teaching people how artificial intelligence systems work, says that “in recent years, there have been significant breakthroughs in empowering computers to understand language just as we do.” This article will give a simple introduction to Natural Language Processing and how it can be achieved. Natural Language Processing, usually shortened as NLP, is a branch of artificial intelligence that deals with the interaction between computers and humans using the natural language. The ultimate objective of NLP is to read, decipher, understand, and make sense of the human languages in a manner that is valuable. Most NLP techniques rely on machine learning to derive meaning from human languages.
"""

In [ ]:
RareWords = rare_words(text)
RareWords

['from',
 'meaning',
 'derive',
 'learning',
 'machine',
 'on',
 'rely',
 'techniques',
 'Most']

In [ ]:
rw_result = remove_fw(text, RareWords)
rw_result

'Natural Language Processing is the technology used to aid computers to understand the human ’ s natural language . It ’ s not an easy task teaching machines to understand how we communicate . Leand Romaf , an experienced software engineer who is passionate at teaching people how artificial intelligence systems work , says that “ in recent years , there have been significant breakthroughs in empowering computers to understand language just as we do. ” This article will give a simple introduction to Natural Language Processing and how it can be achieved . Natural Language Processing , usually shortened as NLP , is a branch of artificial intelligence that deals with the interaction between computers and humans using the natural language . The ultimate objective of NLP is to read , decipher , understand , and make sense of the human languages in a manner that is valuable . NLP to human languages .'

## Conversion of Emoji to Words

To remove or not is done based on the purpose of the application. Example if we are building a sentiment analysis system emoji's can be useful.

"The movie was 🔥"
or
"The movie was 💩"

If we remove the emoji's the meaning of the sentence changes completely. In these cases we can convert emoji's to words.

demoji requires an initial data download from the Unicode Consortium's [emoji code repository](http://unicode.org/Public/emoji/12.0/emoji-test.txt).

On first use of the package, call download_codes().<br>
This will store the Unicode hex-notated symbols at ~/.demoji/codes.json for future use.

Read more about demoji on [pypi.org](https://pypi.org/project/demoji/)

In [ ]:
demoji.download_codes()

... OK (Got response in 1.35 seconds)
Writing emoji data to C:\Users\sagar\.demoji\codes.json ...
... OK


In [ ]:
def emoji_to_words(text):
    return demoji.replace_with_desc(text, sep="__")

In [ ]:
text = "game is on 🔥 🚣🏼"
emoji_to_words(text)

'game is on __fire__ __person rowing boat: medium-light skin tone__'

## Conversion of Emoticons to Words

As we did for emoji's, we convert emoticons to words for the same purpose.

In [ ]:
def emoticons_to_words(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").replace(":","").split()), text)
    return text

In [ ]:
text = "Hey there!! :-)"
emoticons_to_words(text)

'Hey there!! Happy_face_smiley'

## Converting Numbers to Words

If our analysis require us to use information based on the numbers in the text, we can convert them to words.

Read more about num2words on [github](https://github.com/savoirfairelinux/num2words)

In [ ]:
def nums_to_words(text):
    new_text = []
    for word in text.split():
        if word.isdigit():
            new_text.append(num2words(word))
        else:
            new_text.append(word)
    return " ".join(new_text)

In [ ]:
text = "I ran this track 30 times"
nums_to_words(text)

'I ran this track thirty times'

## Chat words Conversion

The more we use social media, we have become lazy to type the whole phrase or word. Due to which slang words came into existance such as "omg" which represents "Oh my god". Such slang words don't provide much information and if we need to use them we have to convert them.

Thank you: [GitHub repo](https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt) for the list of slang words

In [ ]:
#collapse-hide
chat_words = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
QPSA?=Que Pasa?
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
OMG=Oh my god"""

In [ ]:
chat_words_dict = dict()
chat_words_set = set()

def cw_conversion(text):
    new_text = []
    for word in text.split():
        if word.upper() in chat_words_set:
            new_text.append(chat_words_dict[word.upper()])
        else:
            new_text.append(word)
    return " ".join(new_text)

for line in chat_words.split('\n'):
    if line != '':
        cw, cw_expanded = line.split('=')[0], line.split('=')[1]

        chat_words_set.add(cw)
        chat_words_dict[cw] = cw_expanded

In [ ]:
text = "omg that's awesome."
cw_conversion(text)

"Oh my god that's awesome."

## Expanding Contractions

Contractions are words or combinations of words created by dropping a few letters and replacing those letters by an apostrophe.

Example:
- don't: do not
- we'll: we will

Our nlp model don't understand these contractions i.e. they don't understand that "don't" and "do not" are the same thing. If our problem statement requires them then we can expand them or else leave it as it is.

In [ ]:
def expand_contractions(text):
    expanded_text = []
    for line in text:
        expanded_text.append(contractions.fix(line))
    return expanded_text

In [ ]:
text = ["I'll be there within 15 minutes.", "It's awesome to meet your new friends."]
expand_contractions(text)

['I will be there within 15 minutes.',
 'it is awesome to meet your new friends.']

## Stemming

In stemming we reduce the word to it's base or root form by removing the suffix characters from the word. It is one of the technique to normalize text.

Stemming for root word "like" include:
- "likes"
- "liked"
- "likely"
- "liking"

Stemmed word doesn't always match the words in our dictionary such as:
- console -> consol
- company -> compani
- welcome -> welcom

Due to which stemming is not performed in all nlp tasks.

There are various algorithms used for stemming but the most widely used is PorterStemmer. In this post we have used the PorterStemmer as well.

In [ ]:
stemmer = PorterStemmer()

def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [ ]:
df['text_stemmed'] = df.text.apply(lambda text: stem_words(text))
df[['text', 'text_stemmed']].head()

,text,text_stemmed
0,username changed,usernam chang
1,unnieeee,unnieee
2,thanks hope youve got good book keep company,thank hope youv got good book keep compani
3,situated,situat
4,youre welcome im glad liked,your welcom im glad like


PorterStemmer can be used only for english. If we are working with other than english then we can use SnowballStemmer.

In [ ]:
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

## Lemmatization

Lemmatization tried to perform the similar task as that of stemming i.e. trying to reduce the inflection words to it's base form. But lemmatization does it by using a different approach.

Lemmatizations takes into consideration of the morphological analysis of the word. It tries to reduce to words to it's dictionary form which is known as lemma.

In [ ]:
lemmatizer = WordNetLemmatizer()

def text_lemmatize(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df['text_lemmatized'] = df.text.apply(lambda text: text_lemmatize(text))
df[['text', 'text_stemmed', 'text_lemmatized']].head()

,text,text_stemmed,text_lemmatized
0,username changed,usernam chang,username changed
1,unnieeee,unnieee,unnieeee
2,thanks hope youve got good book keep company,thank hope youv got good book keep compani,thanks hope youve got good book keep company
3,situated,situat,situated
4,youre welcome im glad liked,your welcom im glad like,youre welcome im glad liked


Difference between Stemming and Lemmatization:

|Stemming | Lemmatization |
| ---- | ---- |
| Fast compared to lemmatization | Slow compared to stemming |
| Reduces the word to it's base form by removing the suffix | Uses lexical knowledge to get the base form of the word |
| Does not always provide meaning or dictionary form of the original word | Resulting words are always meaningful and dictionary words |

## Spelling Correction

We as human always make mistake. Normally incorrect spelling in text are know as typos.

Since the NLP model doesn't know the difference between a correct and an incorrect word. For the model "thanks" and "thnks" are two different words. Therefore, spelling correction is an important step to bring the incorrect words in the correct format.

In [ ]:
spell = SpellChecker()

def correct_spelling(text):
    correct_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            correct_text.append(spell.correction(word))
        else:
            correct_text.append(word)
    return " ".join(correct_text)

In [ ]:
text = "Hi, hwo are you doin? I'm good thnks for asking"
correct_spelling(text)

"Hi, how are you doing I'm good thanks for asking"

In [ ]:
text = "hw are you doin? I'm god thnks"
correct_spelling(text)

"he are you doing I'm god thanks"

## Convert accented characters to ASCII characters

Accent marks (also referred to as diacritics or diacriticals) usually appear above a character when we press the character for a long time. These need to be remove cause the model cannot distinguish between "dèèp" and "deep". It will consider them as two different words.

In [ ]:
def accented_to_ascii(text):
    return unidecode.unidecode(text)

In [ ]:
text = "This is an example text with accented characters like dèèp lèarning ánd cömputer vísíön etc."
accented_to_ascii(text)

'This is an example text with accented characters like deep learning and computer vision etc.'

## Conclusion

In this article, most of the text pre-processing techniques are explanied. I'll update this post as I learn more techniques to pre-process text.

Share if you liked it, comment if you loved it. Hope to see you guys in the next one. Peace!